In [3]:
%autosave 0

Autosave disabled


In [4]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier
from adam_prepare import titanic_pipeline

Let's read in our data from the titanic pipeline function!

In [5]:
train, val, test = titanic_pipeline()
train.shape, val.shape, test.shape

((623, 10), (134, 10), (134, 10))

We need to create our X and y subsets.

In [8]:
X_train = train.drop(columns = 'survived')
y_train = train.survived

X_val = val.drop(columns = 'survived')
y_val = val.survived

#I am not touching test because you should only test once

In [19]:
#fixing error 'could not convert to male'
X_train = pd.get_dummies(X_train)
X_val = pd.get_dummies(X_val)

X_train.head()

,age,sibsp,parch,fare,alone,sex_female,sex_male,class_First,class_Second,class_Third,deck_A,deck_B,deck_C,deck_D,deck_E,deck_F,deck_G,embark_town_Cherbourg,embark_town_Queenstown,embark_town_Southampton
748,19.0,1,0,53.1000,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,1
45,29.0,0,0,8.0500,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1
28,29.0,0,0,7.8792,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0
633,29.0,0,0,0.0000,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1
403,28.0,1,0,15.8500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1


Before modeling, it's always important to define a baseline!

In [10]:
(y_train == 0 ).mean()   #mean will be accuracy as eval 
#the zero come from y_train.mode() which is the baseline

0.6163723916532905

the majority of the people die on the Titanic. Guessing died everytime has 62% accuracy!

Now we are ready to create a [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) object and start modeling!

In [18]:
X_train.head()

,sex,age,sibsp,parch,fare,class,deck,embark_town,alone
748,male,19.0,1,0,53.1000,First,D,Southampton,0
45,male,29.0,0,0,8.0500,Third,NaN,Southampton,1
28,female,29.0,0,0,7.8792,Third,NaN,Queenstown,1
633,male,29.0,0,0,0.0000,First,NaN,Southampton,1
403,male,28.0,1,0,15.8500,Third,NaN,Southampton,0


In [20]:
seed = 42
rf =RandomForestClassifier(bootstrap=True, 
                            class_weight=None, 
                            criterion='gini',
                            min_samples_leaf=3,
                            n_estimators=100,
                            max_depth=3, 
                            random_state= seed)

rf.fit(X_train, y_train)

RandomForestClassifier(max_depth=3, min_samples_leaf=3, random_state=42)

In [21]:
rf.score(X_train, y_train)

0.8073836276083467

In [22]:
rf.score(X_val, y_val)   
#This is the power of the random forest classifier with zero effort 

0.8208955223880597

In [23]:
#feature selection and feature importances
rf.feature_importances_

array([0.04308948, 0.01466805, 0.01825953, 0.09263582, 0.01424537,
       0.3217283 , 0.30344344, 0.04884606, 0.00660295, 0.08393087,
       0.00217586, 0.002627  , 0.00797881, 0.0115211 , 0.0045106 ,
       0.00271501, 0.        , 0.00919464, 0.00137244, 0.01045468])

In [32]:
fi=pd.DataFrame({'feature': X_train.columns,
             'importance': rf.feature_importances_})
fi.sort_values(by = 'importance', ascending = False)

,feature,importance
5,sex_female,0.321728
6,sex_male,0.303443
3,fare,0.092636
9,class_Third,0.083931
7,class_First,0.048846
0,age,0.043089
2,parch,0.018260
1,sibsp,0.014668
4,alone,0.014245
13,deck_D,0.011521
